In [4]:
from azure.ai.ml import MLClient, command, Input, Output
from azure.ai.ml import load_component
from azure.ai.ml.dsl import pipeline
from azure.ai.ml.constants import AssetTypes
from azure.identity import DefaultAzureCredential


In [3]:
sub_id = "b746917e-ceb7-4ae0-81e6-3ccd893cb0de"
rg = "dpv2"
workspace = "dpv2-wks"

ml_client = MLClient(DefaultAzureCredential(), sub_id, rg, workspace)

In [8]:
from json import load


get_data_func = load_component("./get-data.yml")
train_model_func = load_component("./train-model.yml")
eval_model_func = load_component("./eval-model.yml")


In [11]:
@pipeline()
def train_cifar():
    get_data_step = get_data_func(
        cifar_zip=Input(
            path="wasbs://datasets@azuremlexamples.blob.core.windows.net/cifar-10-python.tar.gz",
            type="uri_file",
        )
    )
    get_data_step.outputs.cifar.mode = "upload"

    train_model_step = train_model_func(
        epochs = 2,
        cifar=get_data_step.outputs.cifar 
    )
    train_model_step.compute = "gpu-cluster"       
    train_model_step.outputs.model_dir.mode = "upload"
    # metang:?: does resources a attributes that every step has or it's just the attributes for this train_model component
    train_model_step.resources.instance_count = 2
    
job = train_cifar()
job.settings.default_compute="cpu-cluster"

In [12]:
ret_job = ml_client.jobs.create_or_update(job, experiment_name="distributed_job_pipeline")

Uploading train-model (0.01 MBs): 100%|##########| 5784/5784 [00:00<00:00, 21992.43it/s]




In [10]:
print(eval_model_func)

$schema: https://azuremlschemas.azureedge.net/latest/commandComponent.schema.json
name: eval_model
display_name: Eval Model
type: command
inputs:
  cifar:
    type: uri_folder
  model_dir:
    type: uri_folder
outputs: {}
command: python main.py --data-dir ${{inputs.cifar}} --model-dir ${{inputs.model_dir}}/model
environment: azureml:AzureML-pytorch-1.9-ubuntu18.04-py37-cuda11-gpu:6
code: ./eval-model
resources:
  instance_count: 2
  properties: {}
distribution:
  type: pytorch
  process_count_per_instance: 2
tags: {}
is_deterministic: true

